<a href="https://colab.research.google.com/github/Likhithavandanachaluvadi/FMML_LABS_23B21A4514/blob/main/FMML_M1L2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine Learning terms and metrics

FMML Module 1, Lab 2

In this lab, we will show a part of the ML pipeline by using the California Housing dataset. There are 20640 samples, each with 8 attributes like income of the block, age of the houses per district etc. The task is to predict the cost of the houses per district. We will use the scikit-learn library to load the data and perform some basic data preprocessing and model training. We will also show how to evaluate the model using some common metrics, split the data into training and testing sets, and use cross-validation to get a better estimate of the model's performance.

In [16]:
import numpy as np
from sklearn import datasets
import matplotlib.pyplot as plt

rng = np.random.default_rng(seed=42)

In [17]:
dataset = datasets.fetch_california_housing()
# Dataset description
print(dataset.DESCR)

.. _california_housing_dataset:

California Housing dataset
--------------------------

**Data Set Characteristics:**

    :Number of Instances: 20640

    :Number of Attributes: 8 numeric, predictive attributes and the target

    :Attribute Information:
        - MedInc        median income in block group
        - HouseAge      median house age in block group
        - AveRooms      average number of rooms per household
        - AveBedrms     average number of bedrooms per household
        - Population    block group population
        - AveOccup      average number of household members
        - Latitude      block group latitude
        - Longitude     block group longitude

    :Missing Attribute Values: None

This dataset was obtained from the StatLib repository.
https://www.dcc.fc.up.pt/~ltorgo/Regression/cal_housing.html

The target variable is the median house value for California districts,
expressed in hundreds of thousands of dollars ($100,000).

This dataset was derived

Given below are the list of target values. These correspond to the house value derived considering all the 8 input features and are continuous values. We should use regression models to predict these values but we will start with a simple classification model for the sake of simplicity. We need to just round off the values to the nearest integer and use a classification model to predict the house value.

In [18]:
print("Orignal target values:", dataset.target)

dataset.target = dataset.target.astype(int)

print("Target values after conversion:", dataset.target)
print("Input variables shape:", dataset.data.shape)
print("Output variables shape:", dataset.target.shape)

Orignal target values: [4.526 3.585 3.521 ... 0.923 0.847 0.894]
Target values after conversion: [4 3 3 ... 0 0 0]
Input variables shape: (20640, 8)
Output variables shape: (20640,)


The simplest model to use for classification is the K-Nearest Neighbors model. We will use this model to predict the house value with a K value of 1. We will also use the accuracy metric to evaluate the model.

In [19]:
def NN1(traindata, trainlabel, query):
    """
    This function takes in the training data, training labels and a query point
    and returns the predicted label for the query point using the nearest neighbour algorithm

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    query: numpy array of shape (d,) where d is the number of features

    returns: the predicted label for the query point which is the label of the training data which is closest to the query point
    """
    diff = (
        traindata - query
    )  # find the difference between features. Numpy automatically takes care of the size here
    sq = diff * diff  # square the differences
    dist = sq.sum(1)  # add up the squares
    label = trainlabel[np.argmin(dist)]
    return label


def NN(traindata, trainlabel, testdata):
    """
    This function takes in the training data, training labels and test data
    and returns the predicted labels for the test data using the nearest neighbour algorithm

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    testdata: numpy array of shape (m,d) where m is the number of test samples and d is the number of features

    returns: the predicted labels for the test data which is the label of the training data which is closest to each test point
    """
    predlabel = np.array([NN1(traindata, trainlabel, i) for i in testdata])
    return predlabel

We will also define a 'random classifier', which randomly allots labels to each sample

In [20]:
def RandomClassifier(traindata, trainlabel, testdata):
    """
    This function takes in the training data, training labels and test data
    and returns the predicted labels for the test data using the random classifier algorithm

    In reality, we don't need these arguments but we are passing them to keep the function signature consistent with other classifiers

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    testdata: numpy array of shape (m,d) where m is the number of test samples and d is the number of features

    returns: the predicted labels for the test data which is a random label from the training data
    """

    classes = np.unique(trainlabel)
    rints = rng.integers(low=0, high=len(classes), size=len(testdata))
    predlabel = classes[rints]
    return predlabel

We need a metric to evaluate the performance of the model. Let us define a metric 'Accuracy' to see how good our learning algorithm is. Accuracy is the ratio of the number of correctly classified samples to the total number of samples. The higher the accuracy, the better the algorithm. We will use the accuracy metric to evaluate and compate the performance of the K-Nearest Neighbors model and the random classifier.

In [21]:
def Accuracy(gtlabel, predlabel):
    """
    This function takes in the ground-truth labels and predicted labels
    and returns the accuracy of the classifier

    gtlabel: numpy array of shape (n,) where n is the number of samples
    predlabel: numpy array of shape (n,) where n is the number of samples

    returns: the accuracy of the classifier which is the number of correct predictions divided by the total number of predictions
    """
    assert len(gtlabel) == len(
        predlabel
    ), "Length of the ground-truth labels and predicted labels should be the same"
    correct = (
        gtlabel == predlabel
    ).sum()  # count the number of times the groundtruth label is equal to the predicted label.
    return correct / len(gtlabel)

Let us make a function to split the dataset with the desired probability. We will use this function to split the dataset into training and testing sets. We will use the training set to train the model and the testing set to evaluate the model.

In [22]:
def split(data, label, percent):
    # generate a random number for each sample
    rnd = rng.random(len(label))
    split1 = rnd < percent
    split2 = rnd >= percent

    split1data = data[split1, :]
    split1label = label[split1]
    split2data = data[split2, :]
    split2label = label[split2]
    return split1data, split1label, split2data, split2label

We will reserve 20% of our dataset as the test set. We will not change this portion throughout our experiments

In [23]:
testdata, testlabel, alltraindata, alltrainlabel = split(
    dataset.data, dataset.target, 20 / 100
)
print("Number of test samples:", len(testlabel))
print("Number of train samples:", len(alltrainlabel))
print("Percent of test data:", len(testlabel) * 100 / len(dataset.target), "%")

Number of test samples: 4144
Number of train samples: 16496
Percent of test data: 20.07751937984496 %


## Experiments with splits

Let us reserve some of our train data as a validation set

In [24]:
traindata, trainlabel, valdata, vallabel = split(
    alltraindata, alltrainlabel, 75 / 100)

What is the accuracy of our classifiers on the train dataset?

In [25]:
trainpred = NN(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Training accuracy using nearest neighbour algorithm:", trainAccuracy*100, "%")

trainpred = RandomClassifier(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Training accuracy using random classifier: ", trainAccuracy*100, "%")

Training accuracy using nearest neighbour algorithm: 100.0 %
Training accuracy using random classifier:  16.4375808538163 %


For nearest neighbour, the train accuracy is always 1. The accuracy of the random classifier is close to 1/(number of classes) which is 0.1666 in our case. This is because the random classifier randomly assigns a label to each sample and the probability of assigning the correct label is 1/(number of classes). Let us predict the labels for our validation set and get the accuracy. This accuracy is a good estimate of the accuracy of our model on unseen data.

In [26]:
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour algorithm:", valAccuracy*100, "%")


valpred = RandomClassifier(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using random classifier:", valAccuracy*100, "%")

Validation accuracy using nearest neighbour algorithm: 34.10852713178294 %
Validation accuracy using random classifier: 16.884689922480618 %


Validation accuracy of nearest neighbour is considerably less than its train accuracy while the validation accuracy of random classifier is the same. However, the validation accuracy of nearest neighbour is twice that of the random classifier. Now let us try another random split and check the validation accuracy. We will see that the validation accuracy changes with the split. This is because the validation set is small and the accuracy is highly dependent on the samples in the validation set. We can get a better estimate of the accuracy by using cross-validation.

In [27]:
traindata, trainlabel, valdata, vallabel = split(
    alltraindata, alltrainlabel, 75 / 100)
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour algorithm:", valAccuracy*100, "%")

Validation accuracy using nearest neighbour algorithm: 34.048257372654156 %


You can run the above cell multiple times to try with different random splits.
We notice that the accuracy is different for each run, but close together.

Now let us compare it with the accuracy we get on the test dataset.

In [28]:
testpred = NN(alltraindata, alltrainlabel, testdata)
testAccuracy = Accuracy(testlabel, testpred)

print("Test accuracy:", testAccuracy*100, "%")

Test accuracy: 34.91795366795367 %


### Try it out for yourself and answer:
1. How is the accuracy of the validation set affected if we increase the percentage of validation set? What happens when we reduce it?
2. How does the size of the train and validation set affect how well we can predict the accuracy on the test set using the validation set?
3. What do you think is a good percentage to reserve for the validation set so that thest two factors are balanced?

Answer for both nearest neighbour and random classifier. You can note down the values for your experiments and plot a graph using  <a href=https://matplotlib.org/stable/gallery/lines_bars_and_markers/step_demo.html#sphx-glr-gallery-lines-bars-and-markers-step-demo-py>plt.plot<href>. Check also for extreme values for splits, like 99.9% or 0.1%

> Exercise: Try to implement a 3 nearest neighbour classifier and compare the accuracy of the 1 nearest neighbour classifier and the 3 nearest neighbour classifier on the test dataset. You can use the KNeighborsClassifier class from the scikit-learn library to implement the K-Nearest Neighbors model. You can set the number of neighbors using the n_neighbors parameter. You can also use the accuracy_score function from the scikit-learn library to calculate the accuracy of the model.

## Multiple Splits

One way to get more accurate estimates for the test accuracy is by using <b>cross-validation</b>. Here, we will try a simple version, where we do multiple train/val splits and take the average of validation accuracies as the test accuracy estimation. Here is a function for doing this. Note that this function will take a long time to execute. You can reduce the number of splits to make it faster.

In [29]:
def AverageAccuracy(alldata, alllabel, splitpercent, iterations, classifier=NN):
    """
    This function takes in the data, labels, split percentage, number of iterations and classifier function
    and returns the average accuracy of the classifier

    alldata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    alllabel: numpy array of shape (n,) where n is the number of samples
    splitpercent: float which is the percentage of data to be used for training
    iterations: int which is the number of iterations to run the classifier
    classifier: function which is the classifier function to be used

    returns: the average accuracy of the classifier
    """
    accuracy = 0
    for ii in range(iterations):
        traindata, trainlabel, valdata, vallabel = split(
            alldata, alllabel, splitpercent
        )
        valpred = classifier(traindata, trainlabel, valdata)
        accuracy += Accuracy(vallabel, valpred)
    return accuracy / iterations  # average of all accuracies

In [30]:
avg_acc = AverageAccuracy(alltraindata, alltrainlabel, 75 / 100, 10, classifier=NN)
print("Average validation accuracy:", avg_acc*100, "%")
testpred = NN(alltraindata, alltrainlabel, testdata)

print("Test accuracy:", Accuracy(testlabel, testpred)*100, "%")

Average validation accuracy: 33.58463539517022 %
Test accuracy: 34.91795366795367 %


This is a very simple way of doing cross-validation. There are many well-known algorithms for cross-validation, like k-fold cross-validation, leave-one-out etc. This will be covered in detail in a later module. For more information about cross-validation, check <a href=https://en.wikipedia.org/wiki/Cross-validation_(statistics)>Cross-validatioin (Wikipedia)</a>

### Questions
1. Does averaging the validation accuracy across multiple splits give more consistent results?Yes, averaging the validation accuracy across multiple splits generally gives more consistent and reliable results. Here's why:

### 1. *Reduction in Variance*
   - When you perform a single train/validation split, the validation accuracy you obtain is highly dependent on the specific data points that end up in the training and validation sets. If the split is particularly favorable or unfavorable (e.g., one split may have more representative or diverse samples, while another may not), the accuracy can vary significantly.
   - By averaging the validation accuracy across multiple random splits, you reduce the impact of any particular split. This process mitigates the variance caused by the randomness of a single split and provides a more stable estimate of the model's performance.

### 2. *Better Generalization Estimate*
   - Averaging across multiple splits gives a better indication of how the model is likely to perform on unseen data. Since each split provides a different perspective by varying the data distribution slightly, the average accuracy reflects the model's performance under varied conditions.
   - This approach helps in assessing how well the model generalizes, which is crucial for predicting its performance on new, unseen data.

### 3. *Comparison with Cross-Validation*
   - This method is a simplified form of cross-validation. In cross-validation, particularly in k-fold cross-validation, the data is split into k subsets, and the model is trained and validated k times, each time with a different subset as the validation set. The final performance metric is the average of these k runs.
   - Both methods aim to achieve the same goal: providing a more robust and reliable estimate of the model's true performance.

### Conclusion:
Averaging validation accuracy across multiple splits indeed gives more consistent and reliable results compared to relying on a single split. This method reduces the variability due to random sampling and helps provide a more accurate estimate of how the model will perform in real-world scenarios.
2. Does it give more accurate estimate of test accuracy?Yes, averaging the validation accuracy across multiple splits can give a more accurate estimate of test accuracy. Here's why:

### 1. *Reduction in Overfitting to a Specific Split*
   - When using a single validation split, the model's performance can be overly optimistic or pessimistic depending on how well the validation set represents the test data. Some splits might inadvertently be more similar to the training set or might contain outliers that skew the results.
   - By averaging across multiple splits, the model's performance is assessed on various subsets of data, reducing the likelihood of overfitting to a specific validation set. This leads to a more balanced and accurate estimate of how the model will perform on the test set.

### 2. *Increased Representativeness*
   - Each split provides a different perspective of the dataset, capturing different patterns and variances in the data. When you average the accuracy across multiple splits, the result is more likely to reflect the overall characteristics of the entire dataset, including the test set.
   - This increased representativeness makes the average validation accuracy a better proxy for the test accuracy.

### 3. *Mitigation of Random Sampling Effects*
   - Random sampling in a single split might lead to a validation set that is not fully representative of the test set, especially in small datasets. Multiple splits smooth out these random effects, providing a more stable and accurate estimate of the model's generalization capability.

### 4. *Better Estimation of Model Performance*
   - Since the validation accuracy across multiple splits averages out the variances and biases inherent in individual splits, it tends to align more closely with the actual test accuracy. This makes it a more reliable indicator of the model's true performance.

### Caveat:
- While averaging validation accuracy across multiple splits gives a better estimate of test accuracy, it is still possible that the test set contains patterns not present in any of the validation sets. For the best estimation, more sophisticated cross-validation techniques like k-fold cross-validation are recommended, where the model is trained and validated on all possible splits of the data.

### Conclusion:
Averaging validation accuracy across multiple splits generally provides a more accurate estimate of the test accuracy, as it reduces the variance caused by individual splits and offers a more comprehensive evaluation of the model's performance on unseen data.
3. What is the effect of the number of iterations on the estimate? Do we get a better estimate with higher iterations?The number of iterations in averaging validation accuracy across multiple splits has a direct impact on the quality of the estimate of test accuracy. Here's how the number of iterations affects the estimate:

### 1. *Reduction in Variance*
   - *Fewer Iterations:* With a small number of iterations, the estimate of validation accuracy can be quite variable. Each iteration may yield a significantly different result, especially if the dataset is small or contains a lot of variability.
   - *More Iterations:* As the number of iterations increases, the variance of the estimate decreases. This means that the average validation accuracy becomes more stable and reliable, converging toward a more accurate representation of the model's performance on unseen data (i.e., test accuracy).

### 2. *Better Sampling of Data Variability*
   - *Fewer Iterations:* With fewer iterations, there is a higher chance that certain data points (or patterns in the data) will be overrepresented or underrepresented in the validation sets. This can lead to an estimate that is not fully representative of the model's general performance.
   - *More Iterations:* Increasing the number of iterations ensures that more data points are included in different validation sets, which better captures the full variability of the data. This leads to a more comprehensive and accurate estimate of the test accuracy.

### 3. *Law of Large Numbers*
   - *Fewer Iterations:* When the number of iterations is low, the average might not fully reflect the true expected value (in this case, the true test accuracy). The estimate may still be influenced by outliers or particular splits that are not representative of the overall dataset.
   - *More Iterations:* According to the law of large numbers, as the number of iterations increases, the average of the results will converge to the expected value (the true underlying test accuracy). This means that with enough iterations, the estimate becomes increasingly accurate.

### 4. *Computational Considerations*
   - *Fewer Iterations:* While fewer iterations are computationally cheaper and faster, they might provide an unreliable estimate.
   - *More Iterations:* Although increasing the number of iterations can be computationally expensive, it leads to a more reliable and accurate estimate. There is, however, a point of diminishing returns where additional iterations provide little to no improvement in the accuracy of the estimate.

### Conclusion:
- *More iterations generally lead to a better estimate* of the test accuracy because they reduce the variance and make the estimate more robust and reliable.
- However, there is a balance to be struck between computational cost and the precision of the estimate. In practice, you might start with a moderate number of iterations and only increase it if the variance in the results suggests that more iterations are necessary to stabilize the estimate.
4. Consider the results you got for the previous questions. Can we deal with a very small train dataset or validation dataset by increasing the iterations?When dealing with a very small training or validation dataset, increasing the number of iterations can help improve the reliability of the model's performance estimate, but it comes with certain limitations. Here's how this works and what challenges might still arise:

### 1. *Improved Estimate Through Averaging*
   - *Increased Iterations:* By increasing the number of iterations, especially in cross-validation, you're effectively using more of the available data for training and validation across different splits. Each iteration might train on slightly different subsets, which helps mitigate the impact of having a small dataset by ensuring that all data points contribute to the training process over the course of multiple iterations.
   - *Averaging:* Averaging the results across these iterations smooths out the effects of outliers or particularly difficult splits. This can lead to a more consistent and potentially more accurate estimate of the model's performance on unseen data, even with a small dataset.

### 2. *Limitations of Small Datasets*
   - *Limited Data Diversity:* If the training dataset is very small, the model may struggle to capture the underlying patterns of the data, regardless of the number of iterations. Each training set might lack diversity, leading to a model that doesn't generalize well. Repeating iterations won't create new data or introduce the diversity needed for robust learning.
   - *Overfitting Risk:* With very small datasets, there's a higher risk of overfitting, where the model learns to perform well on the training data but fails to generalize to new data. Increased iterations won’t necessarily mitigate overfitting; the model may simply memorize the small dataset.

### 3. *Impact on Validation Accuracy*
   - *Validation Set Size:* If the validation set is very small, the accuracy estimates can be highly variable across different splits, even with many iterations. Small validation sets may not be representative of the overall data distribution, leading to unreliable performance estimates. Although averaging across many iterations can reduce this variability, it cannot fully compensate for the lack of data.
   - *Bias in Estimation:* In cases where the validation dataset is too small, the estimates might still be biased, as each validation set might capture only a small portion of the overall variance in the data.

### 4. *Practical Considerations*
   - *Number of Iterations vs. Data Size:* While increasing iterations is helpful, it has diminishing returns when the dataset is very small. There’s a practical limit to how much you can improve the estimate without increasing the actual dataset size.
   - *Data Augmentation:* For very small datasets, it might be beneficial to consider data augmentation techniques or collecting more data, rather than solely relying on more iterations.

### Conclusion:
- *Increased iterations can help to some extent* by providing a more reliable estimate of performance, especially by averaging out the effects of variability in small validation sets.
- *However, increasing iterations alone is not a complete solution* for the challenges posed by very small training or validation datasets. The fundamental issues of data scarcity, lack of diversity, and overfitting require careful consideration, and may need to be addressed through data augmentation, regularization techniques, or acquiring more data.

Ultimately, while more iterations can provide better estimates and mitigate some risks, they cannot fully compensate for the limitations inherent in working with very small datasets.


> Exercise: How does the accuracy of the 3 nearest neighbour classifier change with the number of splits? How is it affected by the split size? Compare the results with the 1 nearest neighbour classifier.